# RAG for cards

In [17]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="qwq:32b")

In [1]:
import json

In [2]:
with open("../data/cards/FDN.json") as f:
    data = json.load(f)

In [3]:
data['data']['cards'][0].keys

<function dict.keys>

In [4]:
from langchain_community.document_loaders import JSONLoader

extraction_data = ['colorIdentity', 'colors', 'convertedManaCost', 'edhrecRank', 
 'keywords', 'legalities', 'manaCost', 'manaValue', 'name',
 'power', 'rarity', 
 'setCode', 'subtypes', 'supertypes', 'text', 
 'toughness', 'type', 'types']

extraction_schema = """
.data.cards[] | {
            colors: .colors,
            convertedManaCost: .convertedManaCost,
            keywords: .keywords,
            manaCost: .manaCost,
            name: .name,
            power: .power,
            rarity: .rarity,
            subtypes: .subtypes,
            supertypes: .supertypes,
            text: .text,
            toughness: .toughness,
            types: .types
        }
"""

loader = JSONLoader(
    file_path="../data/cards/FDN.json",
    jq_schema=extraction_schema,
    text_content=False,
)

In [5]:
docs = loader.load()

In [6]:
print(docs[0]), print(len(docs))

page_content='{"colors": [], "convertedManaCost": 7, "keywords": ["First strike", "Lifelink", "Menace", "Reach", "Trample", "Vigilance", "Ward"], "manaCost": "{7}", "name": "Sire of Seven Deaths", "power": "7", "rarity": "mythic", "subtypes": ["Eldrazi"], "supertypes": [], "text": "First strike, vigilance\nMenace, trample\nReach, lifelink\nWard\u2014Pay 7 life.", "toughness": "7", "types": ["Creature"]}' metadata={'source': '/home/giles/code/deep_mtg/data/cards/FDN.json', 'seq_num': 1}
730


(None, None)

In [7]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="snowflake-arctic-embed2")
vector_store = InMemoryVectorStore(embeddings)

In [8]:
vector_1 = embeddings.embed_query(docs[0].page_content)
vector_2 = embeddings.embed_query(docs[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 1024

[-0.022801735, 0.041194554, 0.017705947, 0.01626768, -0.011525806, -0.0617139, 0.037792053, -0.017419714, -0.022631949, -0.007076665]


In [9]:
len(docs[0].page_content)

392

In [56]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system",
      "You are an expert Magic: The Gathering player."
      "You are helping a new player to understand what various cards do from a high-level perspective."
      "When provided with a card, you should write a concise summary of the card."
      "You can assume that the player understands the basic rules of the game"
      "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
      "Do not include details of rarity or set information."
      "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
      "Include the name of the card in the summary."
      "Include details of the card's role in that, strengths, and weaknesses."
      "Do not return anything other than the summary of the card."),
    ("user", "{card}")]
)

summary = llm.invoke(prompt.invoke({'card': docs[0].page_content})).content
summary

'Sire of Seven Deaths is a strong, flying creature with formidable power and toughness. Its unique ability to ward itself for 7 life points can be a game-changer in the right situations, giving it a significant survival advantage. However, its large converted mana cost makes it difficult to play early on, and its inability to generate card draw or protection for itself may leave it vulnerable to removal spells.'

In [55]:
len(summary)

413

In [25]:
ids = vector_store.add_documents(documents=docs)

In [30]:
vector_store.get_by_ids(['3fd34a5a-9832-4fc6-8f6a-d7384a64bd51'])

[Document(id='3fd34a5a-9832-4fc6-8f6a-d7384a64bd51', metadata={'source': '/home/giles/code/deep_mtg/data/cards/FDN.json', 'seq_num': 1}, page_content='{"colorIdentity": [], "colors": [], "convertedManaCost": 7, "edhrecRank": 8500, "keywords": ["First strike", "Lifelink", "Menace", "Reach", "Trample", "Vigilance", "Ward"], "legalities": {"alchemy": "Legal", "brawl": "Legal", "commander": "Legal", "duel": "Legal", "explorer": "Legal", "future": "Legal", "gladiator": "Legal", "historic": "Legal", "legacy": "Legal", "modern": "Legal", "oathbreaker": "Legal", "pioneer": "Legal", "standard": "Legal", "standardbrawl": "Legal", "timeless": "Legal", "vintage": "Legal"}, "manaCost": "{7}", "manaValue": 7, "name": "Sire of Seven Deaths", "power": "7", "rarity": "mythic", "setCode": "FDN", "subtypes": ["Eldrazi"], "supertypes": [], "text": "First strike, vigilance\\nMenace, trample\\nReach, lifelink\\nWard\\u2014Pay 7 life.", "toughness": "7", "type": "Creature \\u2014 Eldrazi", "types": ["Creatur

In [18]:
results = vector_store.similarity_search(
    "Manacost of omniscience"
)

print(results[0].page_content)

{"colorIdentity": ["U"], "colors": ["U"], "convertedManaCost": 10, "edhrecRank": 1123, "keywords": null, "legalities": {"alchemy": "Legal", "brawl": "Legal", "commander": "Legal", "duel": "Legal", "explorer": "Legal", "future": "Legal", "gladiator": "Legal", "historic": "Legal", "legacy": "Legal", "modern": "Legal", "oathbreaker": "Legal", "pioneer": "Legal", "standard": "Legal", "standardbrawl": "Legal", "timeless": "Legal", "vintage": "Legal"}, "manaCost": "{7}{U}{U}{U}", "manaValue": 10, "name": "Omniscience", "power": null, "rarity": "mythic", "setCode": "FDN", "subtypes": [], "supertypes": [], "text": "You may cast spells from your hand without paying their mana costs.", "toughness": null, "type": "Enchantment", "types": ["Enchantment"]}


In [19]:
results

[Document(id='7a195142-963b-465f-8284-43a6da22dc4d', metadata={'source': '/home/giles/code/deep_mtg/data/cards/FDN.json', 'seq_num': 161}, page_content='{"colorIdentity": ["U"], "colors": ["U"], "convertedManaCost": 10, "edhrecRank": 1123, "keywords": null, "legalities": {"alchemy": "Legal", "brawl": "Legal", "commander": "Legal", "duel": "Legal", "explorer": "Legal", "future": "Legal", "gladiator": "Legal", "historic": "Legal", "legacy": "Legal", "modern": "Legal", "oathbreaker": "Legal", "pioneer": "Legal", "standard": "Legal", "standardbrawl": "Legal", "timeless": "Legal", "vintage": "Legal"}, "manaCost": "{7}{U}{U}{U}", "manaValue": 10, "name": "Omniscience", "power": null, "rarity": "mythic", "setCode": "FDN", "subtypes": [], "supertypes": [], "text": "You may cast spells from your hand without paying their mana costs.", "toughness": null, "type": "Enchantment", "types": ["Enchantment"]}'),
 Document(id='b24d68f1-3457-4584-9d6c-6b42c5d9a030', metadata={'source': '/home/giles/code/

In [10]:
from pathlib import Path

from langchain_community.document_loaders import PyPDFLoader
from langchain_core.tools import tool
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.tools import BaseTool
from typing import Optional, Type

from langchain_core.callbacks import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field

In [11]:
class ScalableQuery(BaseModel):
    query: str = Field(description="Retrieval query")
    k: int = Field(description="Number of results to return")
    score_threshold: float = Field(default=0.0, description="Minimum similarity score to return. Float between zero and one.")

In [24]:
cards = []
for c in docs:
    if 'Exclusion Mage' in c.page_content:
        cards.append(c.page_content)
    if 'Savannah Lions' in c.page_content:
        cards.append(c.page_content)
    if 'Banner of Kinship' in c.page_content:
        cards.append(c.page_content)
    if 'Zetalpa, Primal Dawn' in c.page_content:
        cards.append(c.page_content)
    if 'Etali, Primal Storm' in c.page_content:
        cards.append(c.page_content)
    if 'Mischievous Pup' in c.page_content:
        cards.append(c.page_content)
    if 'Meteor Golem' in c.page_content:
        cards.append(c.page_content)
    if 'Rite of the Dragoncaller' in c.page_content:
        cards.append(c.page_content)
    if 'Suspicious Shambler' in c.page_content:
        cards.append(c.page_content)
    if 'Leonin Skyhunter' in c.page_content:
        cards.append(c.page_content)

In [25]:
cards[0]

'{"colors": ["R"], "convertedManaCost": 6, "keywords": null, "manaCost": "{4}{R}{R}", "name": "Rite of the Dragoncaller", "power": null, "rarity": "mythic", "subtypes": [], "supertypes": [], "text": "Whenever you cast an instant or sorcery spell, create a 5/5 red Dragon creature token with flying.", "toughness": null, "types": ["Enchantment"]}'

In [26]:
import numpy as np

card_idx = np.random.randint(0, len(docs), size=10).tolist()

In [27]:
len(set(cards))

10

In [35]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="deepseek-r1:14b")
summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in set(cards):
    summary = llm.invoke(summary_prompt.invoke({'card': card})).content
    if '</think>' in summary:
        summary = summary[summary.rfind('</think>')+8:]
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    summaru = summary.strip()
    print(summary)
    print(card+'\n')

  **Leonin Skyhunter**: A white Cat Knight creature with flying
It has moderate power and toughness (2/2) for its cost ({W}{W}), making it an affordable yet effective option for aerial strategies
The flying keyword allows it to dodge ground-based blockers, giving it a strong advantage in the sky
However, it is weak against larger creatures or those with reach
As a Cat, it may synergize with other Cat-themed cards.
{"colors": ["W"], "convertedManaCost": 2, "keywords": ["Flying"], "manaCost": "{W}{W}", "name": "Leonin Skyhunter", "power": "2", "rarity": "uncommon", "subtypes": ["Cat", "Knight"], "supertypes": [], "text": "Flying", "toughness": "2", "types": ["Creature"]}

  Mischievous Pup is a white Dog creature with Flash
When it enters the battlefield, you can return up to one other target permanent you control to its owner's hand
It has 3 power and 1 toughness, making it strong offensively but weak defensively
Its low toughness makes it vulnerable, but its ability to recycle cards of

In [47]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="MFDoom/deepseek-r1-tool-calling:8b")
summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in set(cards):
    summary = llm.invoke(summary_prompt.invoke({'card': card})).content
    if '</think>' in summary:
        summary = summary[summary.rfind('</think>')+8:]
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    summaru = summary.strip()
    print(summary)
    print(card+'\n')

  **Leonin Skyhunter** is an uncommon white creature card with a mana cost of {W}{W}
It combines the subtypes of Cat and Knight, offering both combat utility and evasion capabilities
With flying, it can evade most creatures unless they specifically block flying
Its power and toughness of 2 make it balanced in combat, while its ability to bypass obstacles through its Knight subtype enhances its role in evasion strategies
Overall, Leonin Skyhunter is a versatile creature that excels in both direct combat and evasive maneuvers, making it a strong tool for players seeking flexible play options.
{"colors": ["W"], "convertedManaCost": 2, "keywords": ["Flying"], "manaCost": "{W}{W}", "name": "Leonin Skyhunter", "power": "2", "rarity": "uncommon", "subtypes": ["Cat", "Knight"], "supertypes": [], "text": "Flying", "toughness": "2", "types": ["Creature"]}

  **Mischievous Pup**   *Uncommon Creature — Dog Flash (C)*   Mana Cost: {2}{W}   Power/Toughness: 3/1    This playful dog has a flash abilit

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="qwq:32b")
summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in set(cards):
    summary = llm.invoke(summary_prompt.invoke({'card': card})).content
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    print(summary)
    print(card+'\n')

So I've got this card called Etali, Primal Storm
It's a rare creature from Magic: The Gathering, specifically a Legendary Elder Dinosaur
The mana cost is pretty high at {4}{R}{R}, which means you need to pay four colorless mana and two red mana to cast it
So, in terms of mana requirements, it's on the pricier side, making it more of a late-game play
 As for its stats, it's got six power and six toughness, which is pretty decent for a six-mana creature
It can deal a good amount of damage and withstand quite a bit too
However, its real strength lies in its ability rather than just its raw stats
 The ability on Etali is quite unique
Whenever it attacks, it exiles the top card from each player's library, including your own, and then you get to cast any number of those cards without paying their mana costs
This can be incredibly powerful because it allows you to essentially draw cards and play them for free, which can completely swing the game in your favor
 The fact that it exiles the top 

In [18]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="mistral-small")
summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in set(cards):
    summary = llm.invoke(summary_prompt.invoke({'card': card})).content
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    print(summary)
    print(card+'\n')

 **Etali, Primal Storm** - Role: Aggressive beater and card advantage engine
- Strengths: Strong body (6/6), attacks trigger a powerful library exiling effect that can lead to substantial card advantage
- Weaknesses: High mana cost (moderate), vulnerable to removal, and needs to attack consistently to activate its ability
- Mana Cost: Moderately expensive red mana cost.
{"colors": ["R"], "convertedManaCost": 6, "keywords": null, "manaCost": "{4}{R}{R}", "name": "Etali, Primal Storm", "power": "6", "rarity": "rare", "subtypes": ["Elder", "Dinosaur"], "supertypes": ["Legendary"], "text": "Whenever Etali attacks, exile the top card of each player's library, then you may cast any number of spells from among those cards without paying their mana costs.", "toughness": "6", "types": ["Creature"]}

 **Card: Savannah Lions** - Type: Creature (Cat) - Mana Cost: Cheap - Strengths: Strong power for its cost
- Weaknesses: Weak toughness, no abilities or keywords
- Role: Early game aggression.
{"col

In [20]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(model="qwen2.5:32b")
summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in set(cards):
    summary = llm.invoke(summary_prompt.invoke({'card': card})).content
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    print(summary)
    print(card+'\n')

Etali, Primal Storm is aLegendary Elder Dinosaur creature with red mana costing {4}{R}{R}
This card has a strong power and toughness at 6/6
The primary strength of Etali lies in its ability to exile the top cards from each player's library whenever it attacks, allowing you to cast those spells without paying their mana costs
However, this effect could potentially give your opponent powerful cards as well
The high mana cost makes it a late-game play and requires significant red mana to cast.
{"colors": ["R"], "convertedManaCost": 6, "keywords": null, "manaCost": "{4}{R}{R}", "name": "Etali, Primal Storm", "power": "6", "rarity": "rare", "subtypes": ["Elder", "Dinosaur"], "supertypes": ["Legendary"], "text": "Whenever Etali attacks, exile the top card of each player's library, then you may cast any number of spells from among those cards without paying their mana costs.", "toughness": "6", "types": ["Creature"]}

Savannah Lions is a white creature with a weak mana cost of just {W}
It's a

In [43]:
from langchain_core.prompts import ChatPromptTemplate
llm = ChatOllama(model="llama3.1")

summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in [docs[i] for i in card_idx]:
    summary = llm.invoke(summary_prompt.invoke({'card': card.page_content})).content
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    print(summary)
    print(card.page_content+'\n')

Exclusion Mage is a strong blue creature with moderate power and toughness
Its mana cost is relatively low for its strength, making it easy to play early in the game
When it enters the battlefield, it has an ability that allows you to send an opponent's creature back to their hand, disrupting their plans and gaining a strategic advantage
This makes Exclusion Mage a great tool for controlling the board and outmaneuvering opponents.
{"colors": ["U"], "convertedManaCost": 3, "keywords": null, "manaCost": "{2}{U}", "name": "Exclusion Mage", "power": "2", "rarity": "uncommon", "subtypes": ["Human", "Wizard"], "supertypes": [], "text": "When this creature enters, return target creature an opponent controls to its owner's hand.", "toughness": "2", "types": ["Creature"]}

Savannah Lions is a strong, low-cost creature that deals 2 damage and has 1 toughness
Its single white mana cost is relatively weak, but its power makes up for it, making it a good addition to a white-based deck.
{"colors": [

In [44]:
from langchain_core.prompts import ChatPromptTemplate
llm = ChatOllama(model="phi4")

summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in set(cards):
    summary = llm.invoke(summary_prompt.invoke({'card': card})).content
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    print(summary)
    print(card+'\n')

**Leonin Skyhunter**  A white (W) two-mana creature, Leonin Skyhunter is a Cat Knight with Flying
With a power and toughness of 2/2, it serves as an aggressive mid-game threat that can quickly pressure opponents by targeting their creatures or planeswalkers
Its ability to fly makes it difficult to block with ground-based creatures, allowing for strategic attacks that capitalize on its mobility
As a creature that's easy to cast early in the game, Leonin Skyhunter is effective at gaining tempo and maintaining board presence
However, as a 2/2 flier, it can be vulnerable to removal spells or more powerful flying threats later in the game.
{"colors": ["W"], "convertedManaCost": 2, "keywords": ["Flying"], "manaCost": "{W}{W}", "name": "Leonin Skyhunter", "power": "2", "rarity": "uncommon", "subtypes": ["Cat", "Knight"], "supertypes": [], "text": "Flying", "toughness": "2", "types": ["Creature"]}

**Mischievous Pup** is a white flash creature with moderate power and low toughness
The mana cos

In [46]:
from langchain_core.prompts import ChatPromptTemplate
llm = ChatOllama(model="mixtral")

summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in [docs[i] for i in card_idx]:
    summary = llm.invoke(summary_prompt.invoke({'card': card.page_content})).content
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    print(summary)
    print(card.page_content+'\n')

2/2 blue Human Wizard creature with the ability Exclusion Mage when entering the battlefield returns a targeted opponent's creature to its owner's hand
The mana cost is moderate and fits within standard curves
Its role is to disrupt opponents' boards, while its strength lies in its versatility and low cost for this effect
Weaknesses include being vulnerable in combat due to lacking evasion or protection abilities, as well as having no impact if there are no opponent creatures on the battlefield.
{"colors": ["U"], "convertedManaCost": 3, "keywords": null, "manaCost": "{2}{U}", "name": "Exclusion Mage", "power": "2", "rarity": "uncommon", "subtypes": ["Human", "Wizard"], "supertypes": [], "text": "When this creature enters, return target creature an opponent controls to its owner's hand.", "toughness": "2", "types": ["Creature"]}

1-drop White Creature - Savannah Lions  Savannah Lions is a simple yet effective creature for its mana cost
With a power and toughness of 2/1, it's a decent at

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
llm = ChatOllama(model="solar-pro")

summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in [docs[i] for i in card_idx]:
    summary = llm.invoke(summary_prompt.invoke({'card': card.page_content})).content
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    print(summary)
    print(card.page_content+'\n')

Exclusion Mage is a strong uncommon creature card that belongs to the Human Wizard subtype
It has 2 toughness and costs {U}, meaning it requires one blue mana to play
When you cast this spell, it returns an opponent's creature to their hand when it enters the battlefield
This ability can be used strategically to remove threats or disrupt your opponent's plans
However, its power of 2 may not make a significant impact in combat compared to other creatures with higher attack values.
{"colors": ["U"], "convertedManaCost": 3, "keywords": null, "manaCost": "{2}{U}", "name": "Exclusion Mage", "power": "2", "rarity": "uncommon", "subtypes": ["Human", "Wizard"], "supertypes": [], "text": "When this creature enters, return target creature an opponent controls to its owner's hand.", "toughness": "2", "types": ["Creature"]}

The card Saanannah Lions is a white-colored creature with the subtype of cat
It has 2 power and can be destroyed by any amount of damage greater than or equal to 2, making it 

In [47]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI



In [48]:
from langchain_core.prompts import ChatPromptTemplate

ollm = ChatOpenAI(model="gpt-4o-mini")
summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in [docs[i] for i in card_idx]:
    summary = ollm.invoke(summary_prompt.invoke({'card': card.page_content})).content
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    print(summary)
    print(card.page_content+'\n')

Exclusion Mage is a strong blue creature that costs a moderate amount of mana, specifically {2}{U}
With a solid power of 2 and a toughness of 2, it serves as a versatile addition to your deck
Its primary role is as a control card, as it allows you to return a target creature an opponent controls to its owner's hand when it enters the battlefield
This ability can disrupt your opponent's strategy and give you a tactical advantage
However, its relatively low stats mean it can be easily removed if not protected, making it somewhat vulnerable in combat situations
Overall, Exclusion Mage is a valuable tool for maintaining control in the game.
{"colors": ["U"], "convertedManaCost": 3, "keywords": null, "manaCost": "{2}{U}", "name": "Exclusion Mage", "power": "2", "rarity": "uncommon", "subtypes": ["Human", "Wizard"], "supertypes": [], "text": "When this creature enters, return target creature an opponent controls to its owner's hand.", "toughness": "2", "types": ["Creature"]}

Savannah Lions 

In [51]:
from langchain_core.prompts import ChatPromptTemplate

ollm = ChatOpenAI(model="gpt-4o")
summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))

for card in [docs[i] for i in card_idx]:
    summary = ollm.invoke(summary_prompt.invoke({'card': card.page_content})).content
    summary = summary.replace("\n", " ")
    summary = summary.replace('"', "")
    summary = summary.replace(". ", "\n")
    print(summary)
    print(card.page_content+'\n')

Exclusion Mage is a blue creature with a moderate mana cost that fits well in decks focused on disrupting your opponent's board state
When it enters the battlefield, it allows you to temporarily remove an opposing creature by returning it to its owner's hand, which can be effective in breaking momentum or clearing the way for an attack
Its stats are relatively weak, but its ability to disrupt your opponent's plans is its primary strength.
{"colors": ["U"], "convertedManaCost": 3, "keywords": null, "manaCost": "{2}{U}", "name": "Exclusion Mage", "power": "2", "rarity": "uncommon", "subtypes": ["Human", "Wizard"], "supertypes": [], "text": "When this creature enters, return target creature an opponent controls to its owner's hand.", "toughness": "2", "types": ["Creature"]}

Savannah Lions is a white creature card that is characterized by its strong offensive potential for a low mana cost, making it an effective early-game attacker
Despite its weak toughness, which makes it vulnerable to 

In [20]:
from tqdm import tqdm

class CardsRetriever(BaseTool):
    sets_path: Path
    llm: ChatOllama
    embeddings: OllamaEmbeddings
    summary_prompt: ChatPromptTemplate
    card_vector_store: None | InMemoryVectorStore = None
    extraction_schema: str = """
        .data.cards[] | {
            colors: .colors,
            convertedManaCost: .convertedManaCost,
            keywords: .keywords,
            manaCost: .manaCost,
            name: .name,
            power: .power,
            rarity: .rarity,
            subtypes: .subtypes,
            supertypes: .supertypes,
            text: .text,
            toughness: .toughness,
            types: .types
        }
    """
    recreate_storage: bool = False

    name: str = "SetsRetriever"
    description: str = "Provides relevant information for cards in Magic."
    args_schema: Type[BaseModel] = ScalableQuery

    def __init__(self, sets_path: Path, llm:ChatOllama, embeddings: OllamaEmbeddings, recreate_storage: bool = False):
        summary_prompt = ChatPromptTemplate.from_messages((
            [("system",
            "You are an expert Magic: The Gathering player."
            "You are helping a new player to understand what various cards do from a high-level perspective."
            "When provided with a card, you should write a concise summary of the card."
            "You can assume that the player understands the basic rules of the game"
            "Include basic keyword terms like 'flying' or 'trample', but do not explain what they mean."
            "Do not include details of rarity or set information."
            "Rather than quantifying attributes of the card, instead use qualitative terms like 'strong' or 'weak' to describe the card."
            "Include the name of the card in the summary."
            "Include details of the card's role in that, strengths, and weaknesses."
            "Include the mana colors of the card, along with the a qualitative description of the mana cost."
            "Do not return anything other than the summary of the card."
            "Make sure to check that your summary accurately reflects the card."
            ),
            ("user", "{card}")]
        ))
        super().__init__(sets_path=sets_path, llm=llm, embeddings=embeddings, summary_prompt=summary_prompt, recreate_storage=recreate_storage)
        self.create_storage()

    def create_storage(self) -> None:
        if self.recreate_storage or not (self.sets_path/"cards.vec").exists():
            self.card_vector_store = InMemoryVectorStore(self.embeddings)
            for s in self.sets_path.glob("*.json"):
                print(f"Loading {s}...")
                loader = JSONLoader(s, self.extraction_schema, text_content=False)
                cards = loader.load()

                # Remove duplicates and basic lands
                filtered_cards = []
                filtered_hashes = []
                for card in cards:
                    # card_dict = json.loads(card.page_content)
                    # if card_dict["name"] in ["Plains", "Island", "Swamp", "Mountain", "Forest"]:
                    #     continue
                    if (h := hash(card.page_content)) not in filtered_hashes:
                        filtered_cards.append(card)
                        filtered_hashes.append(h)

                # Create card summaries
                print(f"Creating summaries for {len(filtered_cards)} cards...")
                for card in tqdm(filtered_cards):
                    card_dict = json.loads(card.page_content)
                    if "land" in card_dict["types"]:
                        summary = card_dict['text']
                    else:
                        summary = self.llm.invoke(self.summary_prompt.invoke({'card': card.page_content})).content
                    # clean up the summary
                    summary = summary.replace("\n", " ")
                    summary = summary.replace('"', "")
                    card.page_content = '{"summary": "' + summary + '", ' + card.page_content[1:]

                self.card_vector_store.add_documents(documents=filtered_cards)
                print(f"Loaded {len(filtered_cards)} cards from set {s}.")
            
            print(f"Dumping vectors to disk {self.sets_path/"cards.vec"}...")
            self.card_vector_store.dump(self.sets_path/"cards.vec")
        
        else:
            print(f"Loading vectors from disk {self.sets_path/'cards.vec'}...")
            self.card_vector_store = InMemoryVectorStore(self.embeddings).load(self.sets_path/"cards.vec", self.embeddings)

    def _run(
        self, query: str, k: int, score_threshold: float = 0.0, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> list[str]:
        """Retrieve information related to a query."""
        retrieved_cards = self.card_vector_store.similarity_search_with_score(query, k=k)
        if k <= 2:
            score_threshold = 0.0
        filtered_cards = [card[0].page_content for card in retrieved_cards if card[1] > score_threshold]
        return filtered_cards

NameError: name 'BaseTool' is not defined

In [85]:
retriever = CardsRetriever(sets_path=Path("../data/cards"), llm=llm, embeddings=embeddings)

Loading vectors from disk ../data/cards/cards.vec...


In [86]:
retriever.invoke({'query':"Sire of seven deaths", 'k':5, 'score_threshold':0.})

['{"summary": "Sire of Seven Deaths is a strong Eldrazi creature with a high mana cost that\'s considered expensive. It has flying and vigilance, making it a formidable attacker. Its presence on the battlefield allows you to pay 7 life in case you need to save yourself from its wrath, but this also means it can be vulnerable if your opponent finds a way to deal with its threat. Overall, this card is a high-risk, high-reward threat that can swing games in favor of its controller, especially if they have a strong removal spell or a way to protect their life total.", "colors": [], "convertedManaCost": 7, "keywords": ["First strike", "Lifelink", "Menace", "Reach", "Trample", "Vigilance", "Ward"], "manaCost": "{7}", "name": "Sire of Seven Deaths", "power": "7", "rarity": "mythic", "subtypes": ["Eldrazi"], "supertypes": [], "text": "First strike, vigilance\\nMenace, trample\\nReach, lifelink\\nWard\\u2014Pay 7 life.", "toughness": "7", "types": ["Creature"]}',
 '{"summary": "Nine-Lives Famil

In [174]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

memory = MemorySaver()
agent_executor = create_react_agent(llm, [retriever], checkpointer=memory)
config = {"configurable": {"thread_id": "test_cards_retriever"}}

In [175]:
input_message = (
    "What is the mana cost of Omniscience?"
)

for event in agent_executor.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the mana cost of Omniscience?


================================== Ai Message ==================================
Tool Calls:
  SetsRetriever (2118514c-4422-4287-b96c-fbb85e38b68f)
 Call ID: 2118514c-4422-4287-b96c-fbb85e38b68f
  Args:
    k: 1
    query: Omniscience
    score_threshold: 0
================================= Tool Message =================================
Name: SetsRetriever

["{\"colors\": [\"U\"], \"convertedManaCost\": 10, \"keywords\": null, \"manaCost\": \"{7}{U}{U}{U}\", \"manaValue\": 10, \"name\": \"Omniscience\", \"power\": null, \"rarity\": \"mythic\", \"subtypes\": [], \"supertypes\": [], \"text\": \"You may cast spells from your hand without paying their mana costs.\", \"toughness\": null, \"types\": [\"Enchantment\"]}"]
================================== Ai Message ==================================

The mana cost of Omniscience is {7}{U}{U}{U}.


In [176]:
input_message = (
    "How many demons are there? You can retrieve as many cards as you want"
)

for event in agent_executor.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

How many demons are there? You can retrieve as many cards as you want
================================== Ai Message ==================================
Tool Calls:
  SetsRetriever (e8e49fc2-f0cd-43ee-9318-57b3d8cbb0d5)
 Call ID: e8e49fc2-f0cd-43ee-9318-57b3d8cbb0d5
  Args:
    k: 100
    query: demons
    score_threshold: 0
================================= Tool Message =================================
Name: SetsRetriever

["{\"colors\": [\"B\"], \"convertedManaCost\": 3, \"keywords\": null, \"manaCost\": \"{2}{B}\", \"manaValue\": 3, \"name\": \"Infernal Vessel\", \"power\": \"2\", \"rarity\": \"uncommon\", \"subtypes\": [\"Human\", \"Cleric\"], \"supertypes\": [], \"text\": \"When this creature dies, if it wasn't a Demon, return it to the battlefield under its owner's control with two +1/+1 counters on it. It's a Demon in addition to its other types.\", \"toughness\": \"1\", \"types\": [\"Creature\"]}",

In [25]:
from deep_mtg.tools import CardsRetriever
from pathlib import Path
from langchain_ollama import ChatOllama, OllamaEmbeddings
import json

llm = ChatOllama(model="phi4:latest")
embeddings = OllamaEmbeddings(model="snowflake-arctic-embed2")
card_ret = CardsRetriever(sets_path=Path("../data/cards"), llm=llm, embeddings=embeddings)

Creating embeddings for all cards in ../data/cards...


100%|██████████| 13/13 [00:00<00:00, 52078.27it/s]

Loading ../data/cards/BLB.vec...
Loading ../data/cards/MAT.vec...
Loading ../data/cards/LCI.vec...
Loading ../data/cards/BRO.vec...
Loading ../data/cards/OTJ.vec...
Loading ../data/cards/WOE.vec...
Loading ../data/cards/DMU.vec...


Loading ../data/cards/DSK.vec...
Loading ../data/cards/ONE.vec...
Loading ../data/cards/MOM.vec...
Loading ../data/cards/MKM.vec...
Loading ../data/cards/FDN.vec...
Loading ../data/cards/BIG.vec...
Loaded 3616 cards from all sets.


In [22]:
cs = card_ret.invoke({'query':"**Island**: This is a basic land card with no mana cost", 'k':5})
cs

['{"summary": "**Island**  This is a basic land card that requires no mana cost to play.\nIts primary role in your deck is to provide blue mana ({U}), which can be used to cast spells and activate abilities requiring blue mana.\nThe main strength of Island is its ability to generate mana without any additional conditions, making it highly flexible for building various strategies centered around the color blue.\nThere are no inherent weaknesses, but like all lands, it does nothing directly on the battlefield other than produce mana.\nIn decks that heavily rely on blue spells, Islands can be a crucial component in ensuring you have access to necessary resources throughout the game.", "colors": [], "convertedManaCost": 0, "keywords": null, "manaCost": null, "name": "Island", "power": null, "rarity": "common", "subtypes": ["Island"], "supertypes": ["Basic"], "text": "({T}: Add {U}.)", "toughness": null, "types": ["Land"]}',
 '{"summary": "**Island**: This is a basic land card with no mana 

In [29]:
cs[0][86]

'\n'

In [34]:
json.loads(cs[0].replace('\n', " "))

{'summary': '**Island**  This is a basic land card that requires no mana cost to play. Its primary role in your deck is to provide blue mana ({U}), which can be used to cast spells and activate abilities requiring blue mana. The main strength of Island is its ability to generate mana without any additional conditions, making it highly flexible for building various strategies centered around the color blue. There are no inherent weaknesses, but like all lands, it does nothing directly on the battlefield other than produce mana. In decks that heavily rely on blue spells, Islands can be a crucial component in ensuring you have access to necessary resources throughout the game.',
 'colors': [],
 'convertedManaCost': 0,
 'keywords': None,
 'manaCost': None,
 'name': 'Island',
 'power': None,
 'rarity': 'common',
 'subtypes': ['Island'],
 'supertypes': ['Basic'],
 'text': '({T}: Add {U}.)',
 'toughness': None,
 'types': ['Land']}

In [20]:
i = str(card_ret.card_vector_store.store).index('**Island**: This is a basic land card with no mana cost')
c = str(card_ret.card_vector_store.store)[i-100:i+1000]
c = c.replace("\\\\", "")
c = c.replace("\\n", "\n")
c

'022501318, -0.015160546, 0.042740744, 0.007959459, 0.05764019, -0.043312475], \'text\': \'{"summary": "**Island**: This is a basic land card with no mana cost.\nIt serves as a fundamental resource for casting blue spells in your deck.\nWhen tapped (denoted by ({T})), it adds one blue mana (({U})) to your mana pool, allowing you to cast blue spells or activate abilities requiring blue mana.\nIts primary role is to consistently provide mana, making it a versatile and essential component of any blue deck.\nStrengths include its ability to support almost all blue cards; however, since it only produces one type of mana, it lacks flexibility compared to lands that can produce multiple colors.", "colors": [], "convertedManaCost": 0, "keywords": null, "manaCost": null, "name": "Island", "power": null, "rarity": "common", "subtypes": ["Island"], "supertypes": ["Basic"], "text": "({T}: Add {U}.)", "toughness": null, "types": ["Land"]}\', \'metadata\': {\'source\': \'/home/giles/code/deep_mtg/da